In [1]:
import numpy as np
import os
import cv2
import random 
import tensorflow as tf
import pandas as pd

2023-05-01 11:16:09.523035: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 11:16:09.575981: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 11:16:09.577004: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 11:16:10.748124: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
main = "/home/second/Desktop/TB_Chest_Radiography_Database"
normal = "/home/second/Desktop/TB_Chest_Radiography_Database/Normal/"
tuberculosis = "/home/second/Desktop/TB_Chest_Radiography_Database/Tuberculosis/"

df_norm = pd.read_excel("/home/second/Desktop/TB_Chest_Radiography_Database/Normal.metadata.xlsx")
df_tub = pd.read_excel("/home/second/Desktop/TB_Chest_Radiography_Database/Tuberculosis.metadata.xlsx")

In [3]:
df_norm['Tuberculosis'] = np.float(0)
df_tub['Tuberculosis'] = np.float(1)
df_norm.drop(columns = ['FORMAT', 'SIZE', 'URL'], inplace = True)
df_tub.drop(columns = ['FORMAT', 'SIZE', 'URL'], inplace = True)
df = pd.concat([df_norm, df_tub])
df = df.sample(frac=1).reset_index(drop=True)



/tmp/ipykernel_6626/2192205160.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_norm['Tuberculosis'] = np.float(0)
/tmp/ipykernel_6626/2192205160.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_tub['Tuberculosis'] = np.float(1)


In [4]:
df.rename(columns = {'FILE NAME':'Image'}, inplace = True)
label = ['Tuberculosis']

In [ ]:
print("no. of tuberculosis images :" ,len(os.listdir(tuberculosis)))
print("\nno. of normal images :" ,len(os.listdir(normal)))

In [ ]:


tf.random.set_seed(100)
np.random.seed(100)



In [ ]:
train_dir = os.path.join(main, 'train')
test_dir = os.path.join(main, 'test')
validation_dir = os.path.join(main, 'validation')

if not os.path.exists(main):
    os.mkdir(main)
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
if not os.path.exists(test_dir):
    os.mkdir(test_dir)
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
if os.path.exists(validation_dir) or os.path.exists(main) or os.path.exists(train_dir):
    pass

In [ ]:
train_norm_dir = "/home/second/Desktop/TB_Chest_Radiography_Database/train/Normal"
train_tub_dir = "/home/second/Desktop/TB_Chest_Radiography_Database/train/Tuberculosis"

val_norm_dir = "/home/second/Desktop/TB_Chest_Radiography_Database/validation/Normal"
val_tub_dir = "/home/second/Desktop/TB_Chest_Radiography_Database/validation/Tuberculosis"

test_norm_dir = "/home/second/Desktop/TB_Chest_Radiography_Database/test/Normal"
test_tub_dir = "/home/second/Desktop/TB_Chest_Radiography_Database/test/Tuberculosis"

In [ ]:
Labels = ["Normal", "Tuberculosis"]
image_size = 520

train_percentage = 0.8
test_percentage = 0.1
validation_percentage = 0.1

In [ ]:
def create_df(normal_dir, tub_dr ,df):
    df_fn_n = os.listdir(normal_dir)
    df_fn_tb = os.listdir(tub_dr)

    df_fn = df_fn_tb + df_fn_n
    df_ext = []
    for fn in df_fn:
        df_ext.append(fn[:-4])
    df_res = df[df['Image'].isin(df_ext)]
    
    return df_res



In [ ]:
train_df = create_df(train_norm_dir,train_tub_dir,df)
test_df = create_df(test_norm_dir,test_tub_dir,df)
val_df = create_df(val_norm_dir,val_tub_dir,df)


In [ ]:
train_df['Image'] = train_df['Image'] + '.png'
val_df['Image'] = val_df['Image'] + '.png'
test_df['Image'] = test_df['Image'] + '.png'

In [ ]:
print(train_df.shape[0] + val_df.shape[0] + test_df.shape[0])


In [ ]:
import shutil

for cat in Labels:
    cat_dir = os.path.join(main,cat)

    os.makedirs(os.path.join(train_dir, cat))
    os.makedirs(os.path.join(test_dir, cat))
    os.makedirs(os.path.join(validation_dir, cat))

    image_files = [f for f in os.listdir(cat_dir) if f.endswith('.png')]

    random.shuffle(image_files)

    num_images = len(image_files)
    num_train = int(num_images * train_percentage)
    num_test = int(num_images * test_percentage)
    num_validation = num_images - num_train - num_test

    train_files = image_files[:num_train]
    test_files = image_files[num_train:num_train+num_test]
    validation_files = image_files[num_train+num_test:]

    for filename in train_files:
        src = os.path.join(cat_dir, filename)
        dst = os.path.join(train_dir, cat, filename)
        shutil.copyfile(src, dst)

    for filename in test_files:
        src = os.path.join(cat_dir, filename)
        dst = os.path.join(test_dir, cat, filename)
        shutil.copyfile(src, dst)

    for filename in validation_files:
        src = os.path.join(cat_dir, filename)
        dst = os.path.join(validation_dir, cat, filename)
        shutil.copyfile(src, dst)

In [ ]:
def get_train_gen(df,)

In [ ]:

from keras.preprocessing.image import ImageDataGenerator

# imageDataGenerator = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.1,
#     shear_range=0.1,
#     zoom_range=0.1,
#     samplewise_center=True,
#     samplewise_std_normalization=True
# )

In [ ]:
train_gen = imageDataGenerator.flow_from_directory(
    train_dir,
    target_size = (200, 200),
     batch_size = 12,
     shuffle=True,
     class_mode = 'binary',
)
test_gen = imageDataGenerator.flow_from_directory(
    test_dir,
    target_size = (200, 200),
     batch_size = 12,
     shuffle=True,
     class_mode = 'binary',
)
val_gen = imageDataGenerator.flow_from_directory(
    validation_dir,
    target_size = (200, 200),
     batch_size = 12,
     shuffle=True,
     class_mode = 'binary',
)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_gen.classes), y=train_gen.classes)
class_weights


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization



In [ ]:
model=Sequential(name='classifier')
model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(200,200,3),activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(200,200,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))


model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Conv2D(filters=192, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=192, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model.hdf5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max', verbose=1,
    save_best_only=True)

history = model.fit_generator(train_gen,
                              steps_per_epoch=len(train_gen),
                              epochs=10,
                              validation_data=val_gen,
                              validation_steps=len(val_gen),callbacks=[model_checkpoint_callback])

In [ ]:
def predict(image,model):
    labels = ['Normal','Tuberculosis']
    image = np.array(image)
    image = image/image.max()
    image = image.reshape(-1,200,200,3)
    probabilities = model.predict(image).reshape(-1)
    pred = labels[np.argmax(probabilities)]
    return pred, {x:y for x,y in zip(labels, probabilities)}

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


from IPython.display import clear_output
clear_output()


In [ ]:


image = cv2.imread("/home/second/Downloads/Normal-3.png")
image = cv2.resize(image, (200, 200))
pred, probabilities = predict(image, model)

x = list(probabilities.keys())
y = list(probabilities.values())

sns.set_style("darkgrid")
fig, ax = plt.subplots(1,2, figsize=(16,4), gridspec_kw={'width_ratios': [3, 4]})
ax[0].imshow(image)
ax[0].axis('off')

bars = ax[1].barh(x, y, height=0.3, color=['#69c0cd', '#69c0cd'])
ax[1].bar_label(bars)

ax[1].set_title('Predicted Class Probabilities')
plt.xlim([0, 1])
plt.show()

print('Actual   :')
print('Predicted:', pred)
print('-'*80)
print('\n')
